 ESG Finance

 Objectifs
 - Utiliser Python et les librairies usuelles pour analyser de la donnée numérique
 temporelle type série financière;
 - Utiliser Python et les librairies usuelles pour prédire des cours ou des fluc
tuation de cours d’indices boursiers

## Import et Chargement des données

In [89]:
!pip install pandas numpy scikit-learn statsmodels pmdarima matplotlib tensorflow

In [90]:
# les librairies necessaires
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

In [91]:
# 1) Charger les données
url='C:\\Users\\papes\\OneDrive - Ecoles Galiléo Global Education France\\Bureau\\Cours Esg\\machine learning\\Spx_Ml\\data\\raw\\SPX.csv'
df_spx = pd.read_csv(url, sep='\t', header=None)
df_spx=df_spx.rename(columns={0: 'Date', 1: 'Cours'})    # on renomme les colonnes

# 

df_spx = df_spx.sort_values("Date").reset_index(drop=True)
print("Nombre de lignes :",   len(df_spx))  
print(df_spx.head())

Nombre de lignes : 1292
         Date    Cours
0  01/01/1999  1229,23
1  01/01/2010   1115,1
2  01/01/2016  2043,94
3  01/02/2002  1122,19
4  01/02/2008  1395,41


In [92]:
# conversion des dates en format date et les cours en float
df_spx['Date'] = pd.to_datetime(df_spx['Date'], format='%d/%m/%Y') 
df_spx['Cours'] = pd.to_numeric(df_spx['Cours'].str.replace(',', '.'))
df_spx.dtypes

Date     datetime64[ns]
Cours           float64
dtype: object

### Preparation des données

L’objectif est de prédire le cours du SPX à t+1 semaine.

In [93]:
# On suppose que nos données sont quotidiennes (daily).
# On veut prédire le cours à J+5 (5 jours ouvrés plus tard ~ 1 semaine boursière).

df_spx["target"] = df_spx["Cours"].shift(-5)  # pour prédire le cours 5 jours plus tard

On definit les operateurs retards et avance (Close à J-1)

In [ ]:
df_spx["lag1"] = df_spx["Cours"].shift(1) # Valeur retardée de 1 jour
df_spx["lag5"] = df_spx["Cours"].shift(5) # Valeur retardée de 5 jours
df_spx["ma5"]  = df_spx["Cours"].rolling(5).mean() # Moyenne mobile sur 5 jours


df_spx = df_spx.dropna().reset_index(drop=True) # On supprime les premières lignes qui sont NaN
df_spx.head(2)


,Date,Cours,target,lag1,lag5,ma5
0,2013-02-01,1513.17,2803.69,1395.41,1229.23,1437.962
1,2019-02-01,2706.53,1172.92,1513.17,1115.10,1756.248


## Train / Test

On entraîne les modèles avec toutes les données avant le 1er janvier 2012 et on teste après cette date.

In [ ]:
split_date = pd.to_datetime("2012-01-01") # frontière pour séparer les données avant et après 2012:  la date de coupure 


In [96]:
train = df_spx[df_spx["Date"] < split_date] #date est strictement antérieure au 1er janvier 2012.
test  = df_spx[df_spx["Date"] >= split_date] # date est postérieure ou égale au 1er janvier 2012.

In [97]:
# X et y
features = ["Cours", "lag1", "lag5", "ma5" ]
X_train = train[features].values
y_train = train["target"].values

X_test  = test[features].values
y_test  = test["target"].values

print("Taille train:", X_train.shape, "Taille test:", X_test.shape)

Taille train: (879, 4) Taille test: (403, 4)


### Normalisation des données

In [98]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

### Modeles

#### Régression Linéaire

In [99]:
model1 = LinearRegression()
model1.fit(X_train_scaled, y_train)

LinearRegression()

#### Random Forest

In [100]:
model2 = RandomForestRegressor(n_estimators=100, random_state=42)
model2.fit(X_train_scaled, y_train)

RandomForestRegressor(random_state=42)

#### MLPRegressor

In [101]:
model3 = MLPRegressor(hidden_layer_sizes=(50,50),
                      max_iter=1000,
                      random_state=42)
model3.fit(X_train_scaled, y_train)

c:\Users\papes\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(hidden_layer_sizes=(50, 50), max_iter=1000, random_state=42)

## Evaluations et predictions des modeles

#### Predictions

In [102]:
y_pred1 = model1.predict(X_test)
y_pred2 = model2.predict(X_test)
y_pred3 = model3.predict(X_test)

#### Metriques

In [103]:
def evaluate_regression(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return rmse, mae, r2

rmse1, mae1, r2_1 = evaluate_regression(y_test, y_pred1)
rmse2, mae2, r2_2 = evaluate_regression(y_test, y_pred2)
rmse3, mae3, r2_3 = evaluate_regression(y_test, y_pred3)

print("Model 1 (LinearRegression) --> RMSE:", rmse1, "MAE:", mae1, "R²:", r2_1)
print("Model 2 (RandomForest)    --> RMSE:", rmse2, "MAE:", mae2, "R²:", r2_2)
print("Model 3 (MLPRegressor)    --> RMSE:", rmse3, "MAE:", mae3, "R²:", r2_3)

Model 1 (LinearRegression) --> RMSE: 60828.365242106156 MAE: 52364.063140086444 R²: -22814.58584371162
Model 2 (RandomForest)    --> RMSE: 655.4472359254693 MAE: 613.9873677419359 R²: -1.649078375514526
Model 3 (MLPRegressor)    --> RMSE: 1232995.1958294616 MAE: 1214377.9012542756 R²: -9374369.27750951


- Le modèle RandomForest (Modèle 2) a des erreurs en valeur absolue nettement inférieures, ce qui suggère qu'il capte mieux certains aspects des données. Cependant, un R² négatif indique qu'il reste insuffisant pour expliquer la variance des données.
- Les modèles LinearRegression (Modèle 1) et MLPRegressor (Modèle 3) donnent des performances très mauvaises, avec des erreurs énormes et des R² fortement négatifs.

#### Predire la direction 

On cherche à comprendre combien de fois notre modele a predit une augmentation qui a eu lieu mais aussi les hausses qui n'ont eu lieu. Et inversement.

In [104]:
# Direction réelle : 1 si hausse, -1 si baisse
test["real_dir"] = np.where(test["target"] > test["Cours"], 1, -1)

# Pour chaque modèle, on calcule la direction prédite
test["pred1_price"] = y_pred1
test["pred1_dir"] = np.where(test["pred1_price"] > test["Cours"], 1, -1)

test["pred2_price"] = y_pred2
test["pred2_dir"] = np.where(test["pred2_price"] > test["Cours"], 1, -1)

test["pred3_price"] = y_pred3
test["pred3_dir"] = np.where(test["pred3_price"] > test["Cours"], 1, -1)


C:\Users\papes\AppData\Local\Temp\ipykernel_27852\3568850863.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["real_dir"] = np.where(test["target"] > test["Cours"], 1, -1)
C:\Users\papes\AppData\Local\Temp\ipykernel_27852\3568850863.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["pred1_price"] = y_pred1
C:\Users\papes\AppData\Local\Temp\ipykernel_27852\3568850863.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

#### Matrice de confusion

In [105]:
cm1 = confusion_matrix(test["real_dir"], test["pred1_dir"], labels=[1, -1])
print("Confusion matrix Model 1 (rows = true, cols = pred) :\n", cm1)


Confusion matrix Model 1 (rows = true, cols = pred) :
 [[  0  26]
 [ 20 357]]


- 0 : Le modèle n'a jamais prédit "hausse" correctement (vrai positif = 0).
- 26 : Dans 26 cas, le modèle a prédit "baisse" alors que le cours a augmenté (faux négatif = 26).

- 20 : Dans 20 cas, le modèle a prédit "hausse" alors que le cours a baissé (faux positif = 20).
- 357 : Dans 357 cas, le modèle a correctement prédit "baisse" (vrai négatif = 357).

# Conclusion

Bien que le modèle RandomForest affiche des erreurs absolues relativement faibles, aucun des modèles n'explique correctement la variance des données (tous présentent des R² négatifs). Cela suggère que les modèles actuels, tels qu'entraînés, ne sont pas encore adaptés pour prédire efficacement le cours du SPX sur un horizon d'une semaine. Une amélioration significative est nécessaire, que ce soit par une meilleure préparation des données, un enrichissement des features, un ajustement des modèles pour gérer le déséquilibre ou par une optimisation des hyperparamètres. Ces ajustements sont essentiels pour obtenir des modèles de prédiction plus robustes et pertinents dans un contexte financier aussi dynamique.